In [107]:
import pandas as pd


In [108]:
train=pd.read_csv("train.csv")

In [135]:
train=train.dropna()

In [136]:
X=train.drop("label",inplace=False,axis=1)

In [111]:
Y=train["label"]

In [112]:
X.shape

(18285, 4)

In [113]:
import tensorflow as tf

In [114]:
tf.__version__

'2.4.1'

In [124]:
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [116]:
X.reset_index(inplace=True)

In [117]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [118]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YASH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [119]:
re.sub("[^a-zA-Z]"," ",X["title"][0]).split()

['House',
 'Dem',
 'Aide',
 'We',
 'Didn',
 't',
 'Even',
 'See',
 'Comey',
 's',
 'Letter',
 'Until',
 'Jason',
 'Chaffetz',
 'Tweeted',
 'It']

In [120]:
ps=PorterStemmer()
corpus=[]
for i in range(len(X)):
    line=X["title"][i]
    line=re.sub("[^a-zA-Z]"," ",line)
    line=line.lower()
    line=line.split()
    line=[ ps.stem(word) for word in line if not word in stopwords.words("english")]
    corpus.append(" ".join(line))

In [121]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [126]:
voc_size=10000
one_hot_data=[one_hot(words,voc_size) for words in corpus]

In [127]:
one_hot_data

[[7434, 7821, 5232, 6089, 7649, 2451, 337, 6086, 7701, 6554],
 [6737, 9302, 3872, 7179, 987, 7007, 3591],
 [7689, 6165, 1268, 3870],
 [7171, 7841, 6489, 4894, 3114, 8853],
 [1712, 987, 4777, 9608, 1511, 3629, 987, 7845, 1676, 466],
 [5530,
  8158,
  9293,
  5449,
  5625,
  7435,
  6665,
  1081,
  3286,
  9193,
  5632,
  8725,
  2199,
  9875,
  3591],
 [585, 2695, 1407, 8948, 7717, 4893, 1457, 8090, 2422, 4998, 8844],
 [2518, 5358, 7196, 929, 6968, 5155, 7435, 4716, 2422, 4998, 8844],
 [1613, 3107, 9345, 9721, 5591, 607, 8202, 9206, 7435, 1226],
 [4287, 712, 7554, 8392, 2204, 210, 2718, 8981],
 [2950, 559, 3678, 1983, 707, 6202, 388, 4014, 6845, 6716, 5541],
 [4894, 445, 7649, 607, 7435, 6968],
 [2274, 6357, 4104, 3493, 1305, 2036, 4634, 639, 2966],
 [5484, 9640, 6791, 2014, 4805, 9409, 9829, 2422, 4998, 8844],
 [8372, 5908, 1863, 6058, 1424, 2422, 4998, 8844],
 [222, 9693, 4939, 9651, 2199, 4543, 1542, 1978, 8254, 1240],
 [1602, 7146, 9302],
 [6724, 3336, 6841, 7985, 7435, 2577, 5733, 

In [80]:
embedded_docs=pad_sequences(one_hot_data,maxlen=20,padding="pre")

In [81]:
embedded_docs

array([[   0,    0,    0, ..., 6086, 7701, 6554],
       [   0,    0,    0, ...,  987, 7007, 3591],
       [   0,    0,    0, ..., 6165, 1268, 3870],
       ...,
       [   0,    0,    0, ..., 2422, 4998, 8844],
       [   0,    0,    0, ..., 2816,  308, 1673],
       [   0,    0,    0, ..., 5759, 4100, 8641]])

In [96]:
embedded_feature=40
model=Sequential()

In [97]:
model.add(Embedding(voc_size,embedded_feature,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [98]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [99]:
import numpy as np

In [102]:
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [104]:
model.fit(X_final,Y_final,epochs=10,batch_size=64)

Epoch 1/10
286/286 [==============================] - 6s 22ms/step - loss: 0.0041 - accuracy: 0.9989
Epoch 2/10
286/286 [==============================] - 7s 24ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 3/10
286/286 [==============================] - 8s 27ms/step - loss: 0.0020 - accuracy: 0.9994
Epoch 4/10
286/286 [==============================] - 8s 27ms/step - loss: 0.0043 - accuracy: 0.9985
Epoch 5/10
286/286 [==============================] - 7s 24ms/step - loss: 0.0050 - accuracy: 0.9985
Epoch 6/10
286/286 [==============================] - 7s 24ms/step - loss: 0.0041 - accuracy: 0.9989
Epoch 7/10
286/286 [==============================] - 7s 26ms/step - loss: 0.0016 - accuracy: 0.9993
Epoch 8/10
286/286 [==============================] - 7s 24ms/step - loss: 0.0030 - accuracy: 0.9992
Epoch 9/10
286/286 [==============================] - 5s 19ms/step - loss: 6.7958e-04 - accuracy: 0.9998
Epoch 10/10
286/286 [==============================] - 7s 24ms/step - loss: 2.1425e-04 

In [105]:
test_data=pd.read_csv("test.csv")

In [128]:
test_data=test_data.dropna()

In [130]:
test_data.reset_index(inplace=True)

In [138]:
X_test=test_data

In [139]:
corpus_test=[]
for i in range(len(X_test)):
    line=X_test["title"][i]
    line=re.sub("[^a-zA-Z]"," ",line)
    line=line.lower()
    line=line.split()
    line=[ ps.stem(word) for word in line if not word in stopwords.words("english")]
    corpus_test.append(" ".join(line))

In [142]:
one_hot_test=[one_hot(words,voc_size) for words in corpus_test]

In [143]:
one_hot_test

[[6114, 7435, 2171, 5973, 6005, 8756, 9782, 2489, 2422, 4998, 8844],
 [9404, 4478, 9150, 8323, 1045, 3473, 7591, 1104, 4070, 8099],
 [6684, 257, 5573, 5026, 5434, 8844, 5847, 2422, 4998, 8844],
 [1367, 2057, 3247, 7276, 4972],
 [5915, 4266, 1602, 8936, 4805, 2950, 7985, 7435, 3591],
 [3074, 9634, 2456, 1151, 5880],
 [368, 3739, 3767, 2790, 7435, 8843, 1863],
 [6657, 1703, 9623, 763, 2766],
 [9323, 7089, 5788, 9171, 5981, 9714, 9229],
 [1985, 4463, 555, 2880],
 [230, 4914, 3103, 6968, 783, 6357, 6869, 4644, 2618, 7435, 5467, 462, 7180],
 [1376, 5364, 7672, 6341, 3924, 7602, 2030, 2422, 4998, 8844],
 [2422,
  4791,
  5779,
  941,
  7175,
  4434,
  6260,
  4100,
  5232,
  3498,
  7978,
  2422,
  4998,
  8844],
 [5520, 9541, 930, 5289, 7529, 2966, 1761, 3785, 3591],
 [7435, 5242, 3507, 4894, 5404, 1094, 4461, 5743, 3872],
 [7490, 6589, 3438, 6098, 8965, 7422, 6488, 9409, 2422, 4998, 8844],
 [834, 5859, 6087, 8665, 1663, 7597, 2873, 9565],
 [904, 1613, 904, 5743, 7985, 7435, 8522, 7422, 743

In [144]:
embedded_docs_test=pad_sequences(one_hot_test,maxlen=20,padding="pre")

In [145]:
embedded_docs_test

array([[   0,    0,    0, ..., 2422, 4998, 8844],
       [   0,    0,    0, ..., 1104, 4070, 8099],
       [   0,    0,    0, ..., 2422, 4998, 8844],
       ...,
       [   0,    0,    0, ..., 2422, 4998, 8844],
       [   0,    0,    0, ..., 2422, 4998, 8844],
       [   0,    0,    0, ..., 2422, 4998, 8844]])

In [147]:
predict=model.predict_classes(embedded_docs_test)

In [148]:
predict.shape

(4575, 1)

In [152]:
predict

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])